In [0]:
!pip install scikit-optimize
from skopt import gp_minimize
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

df = pd.read_csv('./drive/My Drive/data/esc-50/meta/esc50.csv')
x_mfcc_mean = joblib.load('./drive/My Drive/data/esc-50/x_mfcc_mean.pkl')
x_spec_mean = joblib.load('./drive/My Drive/data/esc-50/x_spec_mean.pkl')
#x_mfcc = joblib.load('./drive/My Drive/data/esc-50/x_mfcc.pkl')
#x_spec = joblib.load('./drive/My Drive/data/esc-50/x_spec.pkl')
#x_mfcc_feat = joblib.load('./drive/My Drive/data/esc-50/x_mfcc_feat.pkl')

X = np.concatenate((x_mfcc_mean, x_spec_mean), axis=1)
y = df['category']

     |████████████████████████████████| 81kB 6.0MB/s 


In [0]:
encode = LabelEncoder()
y = encode.fit_transform(y)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2)

# Utilizando MFCC

In [0]:
rf_mfcc_1 = RandomForestClassifier(max_depth = 72,
                              n_estimators = 1234,
                              min_samples_split = 2,
                              min_samples_leaf = 1,
                              n_jobs=-1)

rf_mfcc_1.fit(X_train[:,:40], y_train)
p_1 = rf_mfcc_1.predict_proba(X_train[:,:40])
p_1_test = rf_mfcc_1.predict_proba(X_test[:,:40])
print(rf_mfcc_1.score(X_test[:,:40], y_test), log_loss(y_test, p_1_test))

0.54 2.3098268418129737


In [0]:
lgbm_params = [0.3971841465687302, 51, 67, 0.9387621172657304, 0.8616798250174156, 1]


lgbm_mfcc_1 = LGBMClassifier(learning_rate=lgbm_params[0], num_leaves=lgbm_params[1], min_child_samples=lgbm_params[2],
                         subsample=lgbm_params[3], colsample_bytree=lgbm_params[4], random_state=0, subsample_freq=lgbm_params[5], 
                         n_estimators=100)

lgbm_mfcc_1.fit(X_train[:,:40], y_train)

p_2 = lgbm_mfcc_1.predict_proba(X_train[:,:40])
p_2_test = lgbm_mfcc_1.predict_proba(X_test[:,:40])

print(lgbm_mfcc_1.score(X_test[:,:40], y_test), log_loss(y_test, p_2_test))

0.505 2.569584667254552


In [0]:
lr_mfcc_1 = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, max_iter=500))
lr_mfcc_1.fit(X_train[:,:40], y_train)
p_3 = lr_mfcc_1.predict_proba(X_train[:,:40])
p_3_test = lr_mfcc_1.predict_proba(X_test[:,:40])
print(lr_mfcc_1.score(X_test[:,:40], y_test), log_loss(y_test, p_3_test))

0.415 2.5801897329627694


In [0]:
xgb_mfcc_1 = XGBClassifier(random_state=0)
xgb_mfcc_1.fit(X_train[:,:40], y_train)
p_4 = xgb_mfcc_1.predict_proba(X_train[:,:40])
p_4_test = xgb_mfcc_1.predict_proba(X_test[:,:40])
print(xgb_mfcc_1.score(X_test[:,:40], y_test), log_loss(y_test, p_4_test))

0.4425 2.1005063765263183


In [0]:
etc_mfcc_1 = ExtraTreesClassifier(n_jobs=-1, random_state=0)
etc_mfcc_1.fit(X_train[:,:40], y_train)
p_5 = etc_mfcc_1.predict_proba(X_train[:,:40])
p_5_test = etc_mfcc_1.predict_proba(X_test[:,:40])
print(etc_mfcc_1.score(X_test[:,:40], y_test), log_loss(y_test, p_5_test))

0.5125 2.7594296251074018


In [0]:
X.shape

(2000, 168)

In [0]:
x_train_nn_mfcc = X_train[:,:40].reshape(1600,40,1)
x_test_nn_mfcc = X_test[:,:40].reshape(400,40,1)

nn_mfcc_1 = Sequential()
nn_mfcc_1.add(Conv1D(64, 2))
nn_mfcc_1.add(Activation('relu'))
nn_mfcc_1.add(Conv1D(64*2, 2))
nn_mfcc_1.add(Activation('relu'))
nn_mfcc_1.add(Conv1D(64*4, 2))
nn_mfcc_1.add(Activation('relu'))
nn_mfcc_1.add(Conv1D(64*8, 2))
nn_mfcc_1.add(Activation('relu'))
nn_mfcc_1.add(Conv1D(64*16, 2))
nn_mfcc_1.add(Activation('relu'))
nn_mfcc_1.add(Flatten())
#nn.add(GlobalMaxPooling1D())
nn_mfcc_1.add(Dense(64*16, activation='relu'))
nn_mfcc_1.add(Dense(64*32, activation='relu'))
nn_mfcc_1.add(Dense(64*64, activation='relu'))
nn_mfcc_1.add(Dense(50))
nn_mfcc_1.add(Activation('softmax'))
nn_mfcc_1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

nn_mfcc_1.fit(x_train_nn_mfcc, y_train, validation_data=(x_test_nn_mfcc, y_test), epochs=100, batch_size=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - 7s 5ms/sample - loss: 4.4569 - acc: 0.0413 - val_loss: 3.6358 - val_acc: 0.0525
Epoch 2/100
1600/1600 [==============================] - 1s 406us/sample - loss: 3.2398 - acc: 0.1112 - val_loss: 2.9725 - val_acc: 0.1400
Epoch 3/100
1600/1600 [==============================] - 1s 405us/sample - loss: 2.7613 - acc: 0.1931 - val_loss: 2.8481 - val_acc: 0.1875
Epoch 4/100
1600/1600 [==============================] - 1s 406us/sample - loss: 2.4524 - acc: 0.2756 - val_loss: 2.4635 - val_acc: 0.3100
Epoch 5/100
1600/1600 [==============================] - 1s 416us/sample - loss: 2.1072 - acc: 0.3731 - val_loss: 2.2878 - val_acc: 0.3225
Epoch 6/100
1600/1600 [==============================] - 1s 415us/sample - loss: 1.7491 - acc: 0.4500 - val_loss: 2.2187 - val_acc: 0.3675
Epoch 7/100
1600/1600 [=================

In [0]:
p_6 = nn_mfcc_1.predict_proba(x_train_nn_mfcc)
p_6_test = nn_mfcc_1.predict_proba(x_test_nn_mfcc)

In [0]:
p_train_mfcc_1 = np.concatenate((p_1, p_2, p_3, p_4, p_5, p_6), axis=1)
p_test_mfcc_1 = np.concatenate((p_1_test, p_2_test, p_3_test, p_4_test, p_5_test, p_6_test), axis=1)

## Segundo nível

In [0]:
lr2_1 = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))
lr2_1.fit(p_train_mfcc_1, y_train)
p2_mfcc_1 = lr2_1.predict_proba(p_train_mfcc_1)
p2_mfcc_1_test = lr2_1.predict_proba(p_test_mfcc_1)
print(lr2_1.score(p_test_mfcc_1, y_test), log_loss(y_test, lr2_1.predict_proba(p_test_mfcc_1)))

0.575 1.8913823856571839


In [0]:
rf2_1 = RandomForestClassifier(random_state=0)
rf2_1.fit(p_train_mfcc_1, y_train)
p2_mfcc_2 = rf2_1.predict_proba(p_train_mfcc_1)
p2_mfcc_2_test = rf2_1.predict_proba(p_test_mfcc_1)
print(rf2_1.score(p_test_mfcc_1, y_test), log_loss(y_test, rf2_1.predict_proba(p_test_mfcc_1)))

0.5275 3.7347215363958073


In [0]:
p_train_mfcc_1.shape

(1600, 300)

In [0]:
x2_train_nn_mfcc = p_train_mfcc_1#.reshape(1600,300,1)
x2_test_nn_mfcc = p_test_mfcc_1#.reshape(400,300,1)

nn_mfcc_2 = Sequential()
nn_mfcc_2.add(Dense(64*16, activation='relu'))
nn_mfcc_2.add(Dense(64*16, activation='relu'))
nn_mfcc_2.add(Dense(64*32, activation='relu'))
nn_mfcc_2.add(Dense(50))
nn_mfcc_2.add(Activation('softmax'))
nn_mfcc_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

nn_mfcc_2.fit(x2_train_nn_mfcc, y_train, validation_data=(x2_test_nn_mfcc, y_test), epochs=10, batch_size=10)

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 1s 464us/sample - loss: 0.5200 - acc: 0.9131 - val_loss: 2.5131 - val_acc: 0.5625
Epoch 2/10
1600/1600 [==============================] - 1s 363us/sample - loss: 2.2435e-05 - acc: 1.0000 - val_loss: 2.4668 - val_acc: 0.5700
Epoch 3/10
1600/1600 [==============================] - 1s 375us/sample - loss: 1.0104e-05 - acc: 1.0000 - val_loss: 2.4785 - val_acc: 0.5700
Epoch 4/10
1600/1600 [==============================] - 1s 382us/sample - loss: 6.3846e-06 - acc: 1.0000 - val_loss: 2.4977 - val_acc: 0.5700
Epoch 5/10
1600/1600 [==============================] - 1s 394us/sample - loss: 4.6503e-06 - acc: 1.0000 - val_loss: 2.5143 - val_acc: 0.5700
Epoch 6/10
1600/1600 [==============================] - 1s 376us/sample - loss: 3.5966e-06 - acc: 1.0000 - val_loss: 2.5308 - val_acc: 0.5700
Epoch 7/10
1600/1600 [==============================] - 1s 376us/sample - loss: 2.8253e-06 - acc: 1.0000 

In [0]:
p2_mfcc_3 = nn_mfcc_2.predict_proba(p_train_mfcc_1)
p2_mfcc_3_test = nn_mfcc_2.predict_proba(p_test_mfcc_1)

In [0]:
p2_mfcc = np.concatenate((p2_mfcc_1, p2_mfcc_2, p2_mfcc_3), axis=1)
p2_mfcc_test = np.concatenate((p2_mfcc_1_test, p2_mfcc_2_test, p2_mfcc_3_test), axis=1)

lr3_spec_mfcc_1 = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))
lr3_spec_mfcc_1.fit(p2_mfcc, y_train)
print(lr3_spec_mfcc_1.score(p2_mfcc_test, y_test))

0.5775


# Utilizando spectrograma

In [0]:
rf_spec_1 = RandomForestClassifier(max_depth = 72,
                                   random_state = 0,
                                   n_estimators = 1234,
                                   min_samples_split = 2,
                                   min_samples_leaf = 1,
                                   n_jobs=-1)

rf_spec_1.fit(X_train[:,40:168], y_train)
p_spec_1 = rf_spec_1.predict_proba(X_train[:,40:168])
p_spec_1_test = rf_spec_1.predict_proba(X_test[:,40:168])
print(rf_spec_1.score(X_test[:,40:168], y_test), log_loss(y_test, p_spec_1_test))

0.3875 2.8752277101925516


In [0]:
lgbm_spec_params = [0.3971841465687302, 51, 67, 0.9387621172657304, 0.8616798250174156, 1]


lgbm_spec_1 = LGBMClassifier(random_state=0,
                             n_estimators=100)

lgbm_spec_1.fit(X_train[:,40:168], y_train)

p_spec_2 = lgbm_spec_1.predict_proba(X_train[:,40:168])
p_spec_2_test = lgbm_spec_1.predict_proba(X_test[:,40:168])

print(lgbm_spec_1.score(X_test[:,40:168], y_test), log_loss(y_test, p_spec_2_test))

0.37 2.8146391093763325


In [0]:
lr_spec_1 = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, max_iter=500))
lr_spec_1.fit(X_train[:,40:168], y_train)
p_spec_3 = lr_spec_1.predict_proba(X_train[:,40:168])
p_spec_3_test = lr_spec_1.predict_proba(X_test[:,40:168])
print(lr_spec_1.score(X_test[:,40:168], y_test), log_loss(y_test, p_spec_3_test))

0.405 2.4167771154546225


In [0]:
xgb_spec_1 = XGBClassifier(random_state=0)
xgb_spec_1.fit(X_train[:,40:168], y_train)
p_spec_4 = xgb_spec_1.predict_proba(X_train[:,40:168])
p_spec_4_test = xgb_spec_1.predict_proba(X_test[:,40:168])
print(xgb_spec_1.score(X_test[:,40:168], y_test), log_loss(y_test, p_spec_4_test))

0.355 2.48015123330988


In [0]:
etc_spec_1 = ExtraTreesClassifier(n_jobs=-1, random_state=0)
etc_spec_1.fit(X_train[:,40:168], y_train)
p_spec_5 = etc_spec_1.predict_proba(X_train[:,40:168])
p_spec_5_test = etc_spec_1.predict_proba(X_test[:,40:168])
print(etc_spec_1.score(X_test[:,40:168], y_test), log_loss(y_test, p_spec_5_test))

0.3725 4.694597036568416


In [0]:
x_train_nn_spec = X_train[:,40:168].reshape(1600,128,1)
x_test_nn_spec = X_test[:,40:168].reshape(400,128,1)

nn_spec_1 = Sequential()
nn_spec_1.add(Conv1D(64, 2))
nn_spec_1.add(Activation('relu'))
nn_spec_1.add(Conv1D(64*2, 2))
nn_spec_1.add(Activation('relu'))
nn_spec_1.add(Conv1D(64*4, 2))
nn_spec_1.add(Activation('relu'))
nn_spec_1.add(Conv1D(64*8, 2))
nn_spec_1.add(Activation('relu'))
nn_spec_1.add(Conv1D(64*16, 2))
nn_spec_1.add(Activation('relu'))
nn_spec_1.add(Flatten())
#nn.add(GlobalMaxPooling1D())
nn_spec_1.add(Dense(64*16, activation='relu'))
nn_spec_1.add(Dense(64*32, activation='relu'))
nn_spec_1.add(Dense(64*64, activation='relu'))
nn_spec_1.add(Dense(50))
nn_spec_1.add(Activation('softmax'))
nn_spec_1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

nn_spec_1.fit(x_train_nn_spec, y_train, validation_data=(x_test_nn_spec, y_test), epochs=100, batch_size=100)

Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - 2s 2ms/sample - loss: 25.7086 - acc: 0.0175 - val_loss: 3.9163 - val_acc: 0.0100
Epoch 2/100
1600/1600 [==============================] - 2s 1ms/sample - loss: 3.8986 - acc: 0.0250 - val_loss: 3.8538 - val_acc: 0.0150
Epoch 3/100
1600/1600 [==============================] - 2s 1ms/sample - loss: 3.8227 - acc: 0.0219 - val_loss: 3.7610 - val_acc: 0.0250
Epoch 4/100
1600/1600 [==============================] - 2s 1ms/sample - loss: 3.7479 - acc: 0.0406 - val_loss: 3.7511 - val_acc: 0.0425
Epoch 5/100
1600/1600 [==============================] - 2s 1ms/sample - loss: 3.6791 - acc: 0.0469 - val_loss: 3.6024 - val_acc: 0.0425
Epoch 6/100
1600/1600 [==============================] - 2s 1ms/sample - loss: 3.5618 - acc: 0.0544 - val_loss: 3.5263 - val_acc: 0.0475
Epoch 7/100
1600/1600 [==============================] - 2s 1ms/sample - loss: 3.4920 - acc: 0.0619 - val_loss: 3.3835 - val_acc: 

In [0]:
p_spec_6 = nn_spec_1.predict_proba(x_train_nn_spec)
p_spec_6_test = nn_spec_1.predict_proba(x_test_nn_spec)

In [0]:
p_spec_train_1  = np.concatenate((p_spec_1, p_spec_2, p_spec_3, p_4, p_spec_5, p_spec_6), axis=1)
p_spec_test_1 = np.concatenate((p_spec_1_test, p_spec_2_test, p_spec_3_test, p_spec_4_test, p_spec_5_test, p_spec_6_test), axis=1)

In [0]:
lr2_spec_1 = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))
lr2_spec_1.fit(p_spec_train_1, y_train)
p2_spec = lr2_spec_1.predict_proba(p_spec_train_1 )
p2_spec_test = lr2_spec_1.predict_proba(p_spec_test_1 )
print(lr2_spec_1.score(p_spec_test_1, y_test), log_loss(y_test, lr2_spec_1.predict_proba(p_spec_test_1)))

0.4125 2.4196581944256894


# mfcc + spec

In [0]:
lr3_spec_mfcc_1 = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))
lr3_spec_mfcc_1.fit(np.concatenate((p2_mfcc, p2_spec), axis=1), y_train)
print(lr3_spec_mfcc_1.score(np.concatenate((p2_mfcc_test, p2_spec_test), axis=1), y_test), log_loss(y_test, lr3_spec_mfcc_1.predict_proba(np.concatenate((p2_mfcc_test, p2_spec_test), axis=1))))

0.55 2.2459402386397067


# Otimização

In [0]:
from sklearn.metrics import log_loss
def treinar_modelo(params):
    learning_rate = params[0]
    num_leaves = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    subsample_freq = params[5]
    
    print(params, '\n')
    
    mdl = LGBMClassifier(learning_rate=learning_rate, num_leaves=num_leaves, min_child_samples=min_child_samples,
                         subsample=subsample, colsample_bytree=colsample_bytree, random_state=0, subsample_freq=subsample_freq, 
                         n_estimators=100)
    
    mdl.fit(p_train, y_train)
    
    return -mdl.score(p_test, y_test)

space = [(1e-3, 1), #learning rate
         (2, 128), # num_leaves
         (1, 100), # min_child_samples
         (0.05, 1.0), # subsample
         (0.1, 1.0), # colsample bytree
         (1, 5)] # colsample bytree

resultados_gp = gp_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=500, n_random_starts=10)



Iteration No: 1 started. Evaluating function at random point.
[0.9971876261279299, 120, 14, 0.9990884895579377, 0.3124800792567785, 3] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.3884
Function value obtained: -0.0250
Current minimum: -0.0250
Iteration No: 2 started. Evaluating function at random point.
[0.3885228304208455, 86, 94, 0.8539953708517165, 0.3819461652390477, 3] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 10.5972
Function value obtained: -0.2250
Current minimum: -0.2250
Iteration No: 3 started. Evaluating function at random point.
[0.4440094408857772, 31, 54, 0.9182639233502715, 0.5114843271882895, 3] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 13.5599
Function value obtained: -0.2375
Current minimum: -0.2375
Iteration No: 4 started. Evaluating function at random point.
[0.9391886616342023, 100, 72, 0.8126196287404969, 0.18352072777666414, 3] 

Iteration No: 4 ended. Evaluation done at random poi

In [0]:
resultados_gp.x

[0.007015920386000394, 128, 22, 0.541256667182959, 0.3454327942071171, 5]

In [0]:
# tentar xgb + rede neural